## Libraries

In [ ]:
import numpy as np
import torch
import math
from cvxopt import matrix, solvers
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris



## Data classes

In [ ]:
class X_values:
    def __init__(self, dataset):
        self.feature_dim = dataset.shape[1]
        self.data_size = dataset.shape[0]
        self.tensor_form = torch.tensor(dataset, dtype=torch.float32)
        print(f"X_values created successfully with dimensions {self.tensor_form.shape}, feature_dim: {self.feature_dim}, data_size: {self.data_size}")

class Y_values:
    def __init__(self, targets):
        self.data_size = targets.shape[0]
        self.tensor_form = torch.tensor(targets, dtype=torch.float32)
        print(f"Y_values created successfully with dimensions {self.tensor_form.shape}")


class Dataset: # !! Use your classes while constructing an instance !!
    def __init__(self, x_values, y_values):
        self.x_tensor = x_values.tensor_form
        self.y_tensor = y_values.tensor_form

        if self.x_tensor.shape[0] != self.y_tensor.shape[0]:
            raise ValueError("Mismatch between X and y dimensions")

        self.feature_dim = x_values.feature_dim
        self.data_size = x_values.data_size

        print(f"Dataset created with X: {self.x_tensor.shape}, Y: {self.y_tensor.shape}")


## Related functions & Kernels

In [ ]:
def split_dataset(dataset, train_rate, val_rate, test_rate):
    if abs(train_rate + val_rate + test_rate - 1.0) > 1e-6:
        raise ValueError("Split rates must sum to 1")

    total_size = dataset.x_tensor.shape[0]
    train_size = int(total_size * train_rate)
    val_size = int(total_size * val_rate)

    indices = torch.randperm(total_size)
    train_indices = indices[:train_size]
    val_indices = indices[train_size:train_size + val_size]
    test_indices = indices[train_size + val_size:]

    x_train, y_train = dataset.x_tensor[train_indices], dataset.y_tensor[train_indices]
    x_val, y_val = dataset.x_tensor[val_indices], dataset.y_tensor[val_indices]
    x_test, y_test = dataset.x_tensor[test_indices], dataset.y_tensor[test_indices]

    train_dataset = Dataset(X_values(x_train.numpy()), Y_values(y_train.numpy()))
    val_dataset = Dataset(X_values(x_val.numpy()), Y_values(y_val.numpy()))
    test_dataset = Dataset(X_values(x_test.numpy()), Y_values(y_test.numpy()))

    return train_dataset, val_dataset, test_dataset

# RBF Kernel Calculator (straightforward)
# (some accelerating/probablistic appraoches are possible, can be implemented in case of such a requirement)
def rbf_kernel(vec_x, vec_y, sigma=1):
    # Assume our vec_x and vec_y are 1-dim torch tensors (vectors), write below in torchized form
    return torch.exp(-(vec_x - vec_y)**2 / (2 * sigma**2))

def polynomial_kernel(vec_x, vec_y, bias=1, deg=3):
  # Assume our vec_x and vec_y are 1-dim torch tensors (vectors), write below in torchized form
  return ((torch.mul(vec_x, vec_y) + bias)**deg)



def rbf_feature_transform(vec_x, sigma=1, expansion_term=4):
    fi_vec_x = []  # Initialize transformed feature vector

    for x_i in vec_x:
        # Compute each feature transformation term
        transformed_value = (
            math.exp(-(x_i**2) / (2 * sigma**2))  # Exponential decay
            * (1 / (math.sqrt(math.factorial(expansion_term)) * sigma**expansion_term))  # Scaling
            * x_i**expansion_term  # Power of the input
        )
        fi_vec_x.append(transformed_value)  # Append to the feature vector

    return fi_vec_x







## SVM Model by Scratch (for now, without kernel)

In [ ]:
def svm_dual_qp(X_tensor, y_tensor, C=1):
    X = X_tensor.numpy().astype(np.float64)  # Ensure float64
    y = y_tensor.numpy().astype(np.float64)  # Ensure float64
    N = X.shape[0]

    # Compute the kernel matrix (linear kernel)
    K = np.dot(X, X.T)

    # Construct QP parameters
    P = matrix(np.outer(y, y) * K)  # Q = y_n * y_m * dot_product(x_n, x_m)
    q = matrix(-np.ones(N))         # Linear term: -1 * sum(alpha_n)
    G = matrix(np.vstack((-np.eye(N), np.eye(N))))  # Inequality: -alpha <= 0 and alpha <= C
    h = matrix(np.hstack((np.zeros(N), np.ones(N) * C)))  # Bounds: 0 <= alpha <= C
    A = matrix(y.reshape(1, -1))   # Equality: sum(y_n * alpha_n) = 0
    b = matrix(0.0)

    # Solve QP problem
    solution = solvers.qp(P, q, G, h, A, b)

    # Extract alpha values
    alphas = np.ravel(solution['x'])
    return torch.tensor(alphas, dtype=torch.float32)

## Testing function (Scratch)

In [ ]:
# Test the implementation using Iris dataset
def test_svm_dual_qp():
    # Load Iris dataset (binary classification: classes 0 and 1)
    iris = datasets.load_iris()
    X = iris.data[iris.target != 2]  # Select only class 0 and 1
    y = iris.target[iris.target != 2]
    y = np.where(y == 0, -1, 1)      # Convert labels to -1 and 1

    # Standardize the dataset
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the dataset
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"ATTENTION: number of data: {len(x_train)}, number of features: {len(x_train[0])}")

    # Create dataset objects
    train_dataset = Dataset(X_values(x_train), Y_values(y_train))
    test_dataset = Dataset(X_values(x_test), Y_values(y_test))

    # Train SVM using dual QP
    alphas = svm_dual_qp(train_dataset.x_tensor, train_dataset.y_tensor)

    # Identify support vectors
    support_vector_indices = torch.where(alphas > 1e-5)[0]
    print("Support vectors:", train_dataset.x_tensor[support_vector_indices])

    # Calculate weight vector and bias
    w = torch.sum(alphas[support_vector_indices][:, None] * train_dataset.y_tensor[support_vector_indices][:, None] * train_dataset.x_tensor[support_vector_indices], dim=0)
    b = train_dataset.y_tensor[support_vector_indices][0] - torch.dot(w, train_dataset.x_tensor[support_vector_indices][0])

    print("Weight vector (w):", w)
    print("Bias (b):", b)

## Scikit-learn SVM Model

In [ ]:
def train_svm_with_sklearn():
    # Load the Iris dataset
    iris = load_iris()
    X = iris.data[iris.target != 2]  # Select only class 0 and 1 for binary classification
    y = iris.target[iris.target != 2]
    y = np.where(y == 0, -1, 1)  # Convert labels to -1 and 1

    # Standardize the dataset
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Split the dataset into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create dataset objects using your classes
    train_dataset = Dataset(X_values(x_train), Y_values(y_train))
    test_dataset = Dataset(X_values(x_test), Y_values(y_test))

    # Train an SVM model using Scikit-Learn
    svm_model = SVC(kernel='linear', C=1.0)  # Linear kernel and penalty parameter C=1.0
    svm_model.fit(train_dataset.x_tensor.numpy(), train_dataset.y_tensor.numpy())

    # Make predictions on the test dataset
    y_pred = svm_model.predict(test_dataset.x_tensor.numpy())

    # Evaluate the model
    accuracy = accuracy_score(test_dataset.y_tensor.numpy(), y_pred)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    # Print support vectors
    print(f"Support vectors:\n{svm_model.support_vectors_}")

    # Print model parameters
    print(f"Weights (w): {svm_model.coef_}")
    print(f"Bias (b): {svm_model.intercept_}")

## Running the code

In [ ]:
# Run the test
print("/////////////////////////")
print("!! SCRATCH MODEL !!")
print("/////////////////////////")
print()
test_svm_dual_qp()
print()

# Run the training and testing function
print("/////////////////////////")
print("!! LIBRARY MODEL !!")
print("/////////////////////////")
print()
train_svm_with_sklearn()
print()

/////////////////////////
!! SCRATCH MODEL !!
/////////////////////////

ATTENTION: number of data: 80, number of features: 4
X_values created successfully with dimensions torch.Size([80, 4]), feature_dim: 4, data_size: 80
Y_values created successfully with dimensions torch.Size([80])
Dataset created with X: torch.Size([80, 4]), Y: torch.Size([80])
X_values created successfully with dimensions torch.Size([20, 4]), feature_dim: 4, data_size: 20
Y_values created successfully with dimensions torch.Size([20])
Dataset created with X: torch.Size([20, 4]), Y: torch.Size([20])
     pcost       dcost       gap    pres   dres
 0: -2.8349e+00 -1.1910e+02  6e+02  2e+00  1e-15
 1: -1.0131e+00 -5.2457e+01  8e+01  2e-01  1e-15
 2:  1.6941e-01 -6.7916e+00  9e+00  2e-02  2e-15
 3: -2.3732e-01 -8.6162e-01  6e-01  2e-04  8e-16
 4: -4.7377e-01 -6.8358e-01  2e-01  4e-05  5e-16
 5: -5.8384e-01 -6.6509e-01  8e-02  3e-06  5e-16
 6: -6.1841e-01 -6.1991e-01  2e-03  5e-08  5e-16
 7: -6.1902e-01 -6.1904e-01  2e-0